In [46]:
! pip install spacy nltk==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/7d/4f/1195bbac8e0c2acc5f740661631d8d750dc38d4a32b23ee5df3cde6f4e0d/joblib-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/fb/13/e3b075031a738c9598c51cfbc4c7879e26729c53aa9cca59211c44235314/regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-an

In [2]:
! python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
input_file = "input/mia's_story.txt"
with open(input_file,encoding="Utf-8") as f:
    text = f.read()
text

'The Lost Kitten\n\nOne sunny afternoon, Mia was playing in her backyard when she heard a tiny "meow." She looked under the bushes and saw a small, gray kitten with white paws. The kitten looked scared and hungry.\n\nMia’s Decision\nMia gently picked up the kitten and asked her mom, "Can we keep it?" Mom smiled and said, "First, we need to check if it belongs to someone." They made posters with the kitten’s picture and their phone number.\n\nThe Search\nFor two days, no one called. Mia named the kitten "Socks" because of its white paws. She gave Socks a cozy bed, a bowl of milk, and a red ball to play with.\n\nA Happy Surprise\nOn the third day, old Mr. Jenkins from down the street called. He said, "That’s my cat, Whiskers!" When Mia brought Socks—now Whiskers—to Mr. Jenkins, he explained that Whiskers had slipped out when the door was open.\n\nA New Friendship\nMr. Jenkins thanked Mia for taking care of Whiskers. He even invited her to visit anytime. Mia was sad but happy Whiskers was

In [38]:
import spacy 
import re 
nlp = spacy.load('en_core_web_sm')

def preprocess(text):
    text = text.replace("\n"," ")
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    sents = []
    for sent in doc.sents:
        noun_count = sum(1 for token in sent if token.pos_ == 'NOUN')
        if noun_count >= 2:
            sents.append(sent.text)
    return sents

In [40]:
sentences = preprocess(text)
sentences

['The Lost Kitten One sunny afternoon, Mia was playing in her backyard when she heard a tiny "meow."',
 'She looked under the bushes and saw a small, gray kitten with white paws.',
 'Mia’s Decision Mia gently picked up the kitten and asked her mom, "Can we keep it?"',
 'They made posters with the kitten’s picture and their phone number.',
 'The Search For two days, no one called.',
 'Mia named the kitten "Socks" because of its white paws.',
 'She gave Socks a cozy bed, a bowl of milk, and a red ball to play with.',
 'On the third day, old Mr. Jenkins from down the street called.',
 'He said, "That’s my cat, Whiskers!"',
 'When Mia brought Socks—now Whiskers—to Mr. Jenkins, he explained that Whiskers had slipped out when the door was open.',
 'A New Friendship Mr. Jenkins thanked Mia for taking care of Whiskers.',
 'The next week, Mom surprised Mia with a new kitten from the animal shelter!']

In [27]:
import random
def select_random_sents(sentences,max_choice=5):
    random_sents = random.sample(sentences,max_choice)
    return random_sents

In [ ]:
chosen_sents = select_random_sents(sentences)
chosen_sents

['Mia named the kitten "Socks" because of its white paws.',
 'She gave Socks a cozy bed, a bowl of milk, and a red ball to play with.',
 'A New Friendship Mr. Jenkins thanked Mia for taking care of Whiskers.',
 'The next week, Mom surprised Mia with a new kitten from the animal shelter!',
 'They made posters with the kitten’s picture and their phone number.']

In [45]:
#Whole text NER lists 
ner_dict = {}
all_text = " ".join(sentences)
doc = nlp(all_text)
for ent in doc.ents:
    if ent.label_ not in ner_dict:
        ner_dict[ent.label_] = []
    ner_dict[ent.label_].append(ent.text)
ner_dict

{'TIME': ['One sunny afternoon'],
 'PERSON': ['Mia',
  'Mia’s Decision Mia',
  'Mia',
  'Jenkins',
  'Mia',
  'Jenkins',
  'Jenkins',
  'Mia',
  'Mom',
  'Mia'],
 'DATE': ['two days', 'the third day', 'The next week'],
 'ORG': ['Whiskers', 'Whiskers', 'Whiskers', 'Whiskers']}

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

def get_distractors_wordnet(syn, word):
    distractors = []
    word = word.lower().replace(" ", "_")
    orig_word = word
    hypernyms = syn.hypernyms()
    if not hypernyms:
        return distractors
    for item in hypernyms[0].hyponyms():
        name = item.lemmas()[0].name()
        if name == orig_word:
            continue
        name = " ".join(w.capitalize() for w in name.replace("_", " ").split())
        if name and name not in distractors:
            distractors.append(name)
    return distractors



In [55]:
original_word = "Whiskers"
synset = wn.synsets(original_word, 'n')[0]
synset

Synset('beard.n.01')

In [56]:
distractors = get_distractors_wordnet(synset, original_word)
distractors

['Beard', 'Mustache', 'Sideburn']